# 15 基于mindspore实现多分类损失函数

二分类损失函数中定义了一个简单的平均绝对误差损失函数MAELoss，但许多深度学习应用的数据集较复杂，如目标检测网络Faster R-CNN的数据中就包含多个标签，而非简单的一条数据对应一个标签，这时损失函数的定义和使用略有不同。

本节介绍在多标签数据集场景下，如何定义多标签损失函数（Multi label loss function），并使用Model进行模型训练。

# 15.1 构建多标签数据集

下例通过 get_multilabel_data 函数拟合两组线性数据 $y 1$ 和 $y 2$ ，拟合的目标函数为:
$$
f(x)=2 x+3
$$
最终数据集应随机分布于函数周边，这里按以下公式的方式生成，其中 noise 为服从标准正态分布的随机值。 get_multilabel_data函数返 回数据 $x 、 y 1$ 和 $y 2$ :
$$
f(x)=2 x+3+\text { noise }
$$
通过 create_multilabel_dataset 生成多标签数据集，并将 GeneratorDataset 中的 column_names 参数设置为['data', 'label1', 'label2']，最终返 回的数据集格式为一条数据” data ”对应两个标签 label1 和 labe12 。

In [4]:
import numpy as np
from mindspore import dataset as ds

def get_multilabel_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise1 = np.random.normal(0, 1)
        noise2 = np.random.normal(-1, 1)
        y1 = x * w + b + noise1
        y2 = x * w + b + noise2
        yield np.array([x]).astype(np.float32), np.array([y1]).astype(np.float32), np.array([y2]).astype(np.float32)

def create_multilabel_dataset(num_data, batch_size=16):
    dataset = ds.GeneratorDataset(list(get_multilabel_data(num_data)), column_names=['data', 'label1', 'label2'])
    dataset = dataset.batch(batch_size)  # 每个batch有16个数据
    return dataset


# 15.2 多标签损失函数

针对上一步创建的多标签数据集，定义多标签损失函数 MAELossForMultiLabel 。
$$
\begin{aligned}
\operatorname{los} s 1 & =\frac{1}{m} \sum_{i=1}^m\left|y 1_i-f\left(x_i\right)\right| \\
\operatorname{los} 2 & =\frac{1}{m} \sum_{i=1}^m\left|y 2_i-f\left(x_i\right)\right| \\
\operatorname{loss} & =\frac{(\operatorname{los} s 1+\operatorname{loss} 2)}{2}
\end{aligned}
$$
上式中， $f(x)$ 为样例标签的预测值， $y 1$ 和 $y 2$ 为样例标签的真实值， loss 1 为预测值与真实值 $y 1$ 之间距离的平均值， loss 2 为预测值与 真实值 $y 2$ 之间距离的平均值，loss 为损失值 loss 1 与损失值 loss 2 平均值。
在 MAELossForMultilabel 中的 construct 方法的输入有三个，预测值 base，真实值 target1 和 target2 ，在 construct 中分别计算预测值 与真实值 target1 、预测值与真实值 target2 之间的误差，将两误差取平均后作为最终的损失函数值。


In [5]:
class MAELossForMultiLabel(nn.LossBase):
    def __init__(self, reduction="mean"):
        super(MAELossForMultiLabel, self).__init__(reduction)
        self.abs = ops.Abs()

    def construct(self, base, target1, target2):
    '''
    偏差
    '''
        return (self.get_loss(x1) + self.get_loss(x2))/2


In [ ]:
#参考答案
    x1 = self.abs(base - target1)
    x2 = self.abs(base - target2)

# 15.3 多标签模型训练 
使用 Model 关联指定的前向网络、损失函数和优化器时，因 Model 内默认使用的 nn.WithLossCell 只接受两个输入： data 和 label ，故 不适用于多标签场景。
在多标签场景下，若想使用 Mode1 进行模型训练，则需事先把前向网络与多标签损失函数关联起来，即自定义损失网络。
- 定义损失网络
下例展示了如何定义损失网络 CustomWithLossCell，其中 __init_ 方法的两个参数 backbone 和 loss_fn 分别表示前向网络和损失函数， construct 方法的输入分别为样例输入 data 和样例真实标签 label1、 labe12，将样例输入 data 传给前向网络 backbone，将预测值和 两标签值传给损失函数 loss_fn 。

In [6]:
class CustomWithLossCell(nn.Cell):

    '''
    构造函数
    返回值有 output, label1, label2
    '''


In [ ]:
#参考答案
    def __init__(self, backbone, loss_fn):
        super(CustomWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn

    def construct(self, data, label1, label2):
        output = self._backbone(data)
        return self._loss_fn(output, label1, label2)

# 15.4 定义网络模型并训练
使用Model连接前向网络、多标签损失函数和优化器时，Model的网络network指定为自定义的损失网络loss_net，损失函数loss_fn不指定，优化器仍使用Momentum。

在未指定loss_fn时，Model会默认network内部已实现损失函数的逻辑，不再在内部使用nn.WithLossCell关联前向网络和损失函数。

In [11]:
from mindspore.common.initializer import Normal
import mindspore.ops as ops
from mindvision.engine.callback import LossMonitor
class LinearNet(nn.Cell):
    """定义线性回归网络"""
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))

    def construct(self, x):
        return self.fc(x)
    
ds_train = create_multilabel_dataset(num_data=160)
net = LinearNet()

# 定义多标签损失函数
loss = MAELossForMultiLabel()

# 定义损失网络，连接前向网络和多标签损失函数
loss_net = CustomWithLossCell(net, loss)

# 定义优化器
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)

# 定义Model，多标签场景下Model无需指定损失函数
model = ms.Model(network=loss_net, optimizer=opt)

model.train(epoch=1, train_dataset=ds_train, callbacks=[LossMonitor(0.005)])


[WARNING] ME(5924:12492,MainProcess):2022-12-23-16:01:10.939.243 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.


Epoch:[  0/  1], step:[    1/   10], loss:[11.166/11.166], time:882.640 ms, lr:0.00500
Epoch:[  0/  1], step:[    2/   10], loss:[11.010/11.088], time:17.951 ms, lr:0.00500
Epoch:[  0/  1], step:[    3/   10], loss:[10.315/10.830], time:0.999 ms, lr:0.00500
Epoch:[  0/  1], step:[    4/   10], loss:[11.732/11.056], time:3.002 ms, lr:0.00500
Epoch:[  0/  1], step:[    5/   10], loss:[9.359/10.717], time:4.988 ms, lr:0.00500
Epoch:[  0/  1], step:[    6/   10], loss:[6.328/9.985], time:0.998 ms, lr:0.00500
Epoch:[  0/  1], step:[    7/   10], loss:[7.772/9.669], time:1.994 ms, lr:0.00500
Epoch:[  0/  1], step:[    8/   10], loss:[6.167/9.231], time:5.984 ms, lr:0.00500
Epoch:[  0/  1], step:[    9/   10], loss:[9.109/9.218], time:2.992 ms, lr:0.00500
Epoch:[  0/  1], step:[   10/   10], loss:[5.866/8.883], time:0.996 ms, lr:0.00500
Epoch time: 955.423 ms, per step time: 95.542 ms, avg loss: 8.883
